In [1]:
import numpy as np


# 関数を改良
class Variable:
    def __init__(self, data):
        # 追加
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError("{} is not supported".format(type(data)))
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        # 追加
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input
        output.set_creator(self)
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        return np.exp(self.input.data) * gy


# 追加
def square(x):
    return Square()(x)

def exp(x):
    return Exp()(x)

In [2]:
x = Variable(np.array(1.0))

In [3]:
x = Variable(1.0)

TypeError: <class 'float'> is not supported

In [4]:
# numpyの問題点
# 問題ない場合
x = np.array([1.0])
y = x ** 2
print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 1
<class 'numpy.ndarray'>


In [6]:
# 問題ある場合
x = np.array(1.0)
y = x ** 2
print(type(x), x.ndim)
print(type(y))

# このようにスカラーを演算すると不動小数点型に変化してしまう

<class 'numpy.ndarray'> 0
<class 'numpy.float64'>


In [7]:
# 独自の関数を用意
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    else:
        return x

In [8]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()